In [ ]:
# preprocess a dataset
def create_qqp(split=0.9):
    x = pd.read_csv("./data/annotated/qqp/questions.csv")
    q1, q2 = list(x.question1), list(x.question2)
    labels = list(x.is_duplicate)
    train, test, valid = [], [], []
    trainl, testl, validl = (int)(0.9 * len(q1)), (int)(0.05 * len(q1)), (int)(0.05 * len(q1))

    for i in range(trainl):
        try:
            train.append(clean_line(q1[i]) + ' . ' + clean_line(q2[i]) + ' , ' + str(labels[i]) + '\n')
        except:
            continue
        
    for i in range(trainl, trainl+testl, 1):
        try:
            test.append(clean_line(q1[i]) + ' . ' + clean_line(q2[i]) + ' , ' + str(labels[i]) + '\n')
        except:
            continue
    
    for i in range(trainl+testl, len(q1), 1):
        try:
            valid.append(clean_line(q1[i]) + ' . ' + clean_line(q2[i]) + ' , ' + str(labels[i]) + '\n')
        except:
            continue

    def write_to_disk(lines, output):
        txt = open(output, 'w')
        for line in lines:
            txt.write(line.lower())
    
    write_to_disk(train, './data/annotated/qqp/train.csv')
    write_to_disk(test, './data/annotated/qqp/test.csv')
    write_to_disk(valid, './data/annotated/qqp/valid.csv')


In [ ]:
#fix references in style-masked/yelp to make sure it aligns with test.txt
from utils import clean_line
lbls = ['0','1']
datasets = ['amazon-cleaned']
for d in datasets:
    for lbl in lbls:
        dst = '1' if lbl == '0' else '0'
        path1 = './data/style-masked/{}/{}/reference.{}to{}'.format(d, lbl, lbl, dst)
        m = {}
        with open(path1, 'r') as f:
            for line in f:
                base, human = line.split('\t')
                base = clean_line(base)
                base = ' '.join(base.split())
                m[base] = human
        path2 = './data/style-masked/{}/{}/reference.{}to{}.cleaned'.format(d, lbl, lbl, dst)
        path3 = './data/style-masked/{}/{}/test.txt'.format(d, lbl)
        
        with open(path2, 'w') as f1:
            with open(path3, 'r') as f2:
                for line in f2:
                    line = clean_line(line)
                    line = ' '.join(line.split())
                    f1.write(m[line])

In [ ]:
# run nlg evaluation for SMLM models. produces nlg_evals_humans
from nlgeval import compute_metrics
def nlg_eval_human(model_name, dataset):
    baseline = os.path.join('checkpoints', 'style-masked', dataset, model_name, 'exps_model_best.pt')
    avg_metrics = {}
    lbls = ['0', '1']
    for src_lbl in lbls:
        dst_lbl = '1' if src_lbl == '0' else '0'	
        path1 = os.path.join(baseline, 'test.{}to{}'.format(src_lbl, dst_lbl))
        path2 = os.path.join("./data/style-masked/{}/{}".format(dataset, src_lbl), 'reference.{}to{}.cleaned'.format(src_lbl, dst_lbl))
        print("performing nlg eval between {} and {}".format(path1, path2))
        metrics = compute_metrics(hypothesis=path1, references=[path2], no_skipthoughts=True, not_naturalness=True)
        print(metrics)
        with open(path1+'.nlg-evals', 'w') as f:
            for k in metrics:
                if(k not in avg_metrics):
                    avg_metrics[k] = metrics[k]
                else:
                    avg_metrics[k] += metrics[k]
                f.write('{}: {} \n'.format(k, metrics[k]))
    with open(os.path.join(baseline, 'nlg_evals_human'), 'w') as f:
        for k in metrics:
            f.write('{}: {} \n'.format(k, avg_metrics[k]/2))


In [ ]:
datasets = ['amazon-cleaned']
for dataset in datasets:
    nlg_eval_human('smlmg-c10-eps0.15', dataset)

In [ ]:
lbls = ['0', '1']

for lbl in lbls:
    dst_lbl = '0' if lbl == '1' else '1'
    cleaned_test = './data/annotated/amazon-cleaned/sentiment.test.{}'.format(lbl)
    full_refs = './data/annotated/amazon-cleaned/references.{}to{}'.format(lbl, dst_lbl)

    m = {}

    with open(full_refs, 'r') as f:
        for line in f:
            base, human = line.split('\t')
            base = clean_line(base)
            base = ' '.join(base.split())
            m[base] = human

    for i, line in enumerate(m):
        if(i==5):
            break
        print(line)
    
    cleaned_refs = './data/annotated/amazon-cleaned/references.{}to{}.cleaned'.format(lbl, dst_lbl)

    with open(cleaned_test, 'r') as f1:
        with open(cleaned_refs, 'w') as f2:
            for line in f1:
                old_line = line
                line = clean_line(line)
                line = ' '.join(line.split())
                f2.write(old_line + '\t' + m[line])

    

In [ ]:
# get acciracy% metrics for attri method comparison table
import argparse
import os
import matplotlib.pyplot as plt
from MulticoreTSNE import MulticoreTSNE as TSNE
import torch
from tqdm import tqdm
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from nltk.translate.bleu_score import sentence_bleu 
from vocab import Vocab
from model import *
from utils import *
from preproc import create_toy_dataset
from batchify import *
from nlgeval import compute_metrics
import nltk

def load_sent(path): # used to load sents from unannotated datasets with format: {input}
	inputs = []
	with open(path) as f:
		for line in f:
			line = line.split(',')[0]
			inputs.append(line.split())
	return inputs

def classify(sents, model, args, batch_size=32):
	# assert(args.classifier_checkpoint != None)
	print("Predicting labels for {} sentences...".format(len(sents)))
	batches, _ = get_batches(sents, classifier_vocab, batch_size, torch.device('cpu'), sort=False)
	preds = []
	for i, (inputs, _) in enumerate(batches):
		_, _, outs = model(inputs, args)  # BO
		outs = torch.argmax(outs, dim=-1)  # B1
		preds.append(outs.detach().cpu().numpy())
	preds = np.concatenate(preds, axis=0)
	return preds

def get_model(path, vocab):
	print("loading model from path {}".format(path))
	ckpt = torch.load(path)
	train_args = ckpt['args']    
	model = {'dae': DAE, 'vae': VAE, 'aae': AAE, 'van': VanillaAE, 
		'lstmclassifier': LSTMClassifier, 'divlstmclassifier':DiversityLSTMClassifier, 
		'tr-encoder':TransformerModel}[ 
		train_args.model_type](vocab, train_args).to(torch.device('cpu'))
	model.load_state_dict(ckpt['model'])
	if(not isinstance(model, TransformerModel)):
		model.flatten()
	return model, train_args

def trythis(data, classifier_model, classifier_args, lbl):
    sents = load_sent(data)
    preds = classify(sents, classifier_model, classifier_args)
    wrong = 0
    acc = 0
    with open(data+'.preds', 'w') as f:
        for i, p in enumerate(preds):
            if(p != lbl):
                wrong += 1
            f.write(' '.join(sents[i]) + '\t' + str(p) + '\n')
        acc = (len(preds) - wrong)/len(preds)
        f.write('Style Transfer Accuracy: {} \n'.format(str(acc)))
    return acc

dataset = 'snli'
r = [0.0,0.1,0.2,0.3,0.4,0.5]
for i in r:
	ckpt = 'snli-{}'.format(i)
	classifier_checkpoint = './checkpoints/supervised/{}/classifier'.format(dataset)
	if(classifier_checkpoint != None): #used in TST exp mainly
		classifier_vocab = Vocab(os.path.join(classifier_checkpoint, 'vocab.txt'))
		classifier_model, classifier_train_args = get_model(os.path.join(classifier_checkpoint, 'checkpoints', 'model_best.pt'), classifier_vocab)
	lbls = ['0','1']
	m = 0
	for lbl in lbls:
		data = './data/style-masked/{}/{}/test.csv'.format(ckpt, lbl)
		acc = trythis(data, classifier_model, classifier_train_args, int(lbl))
		m += acc
	print(m/2)


In [ ]:
#plot line graphs for eps analysis exp
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams['text.usetex'] = True
plt.rcParams.update({'font.size': 15})

figure, a = plt.subplots(2, figsize=(10,10))
x = [0.0,0.1,0.2,0.3,0.4,0.5]
yelp = [55,62,66,70,72,75]
imdb = [float(e) for e in "54	65	73	78	81	84".split()]
amazon = [float(e) for e in "48	67	78	84	88	91".split()]
snli = [float(e) for e in "56	65	72	78	82	85".split()]
a[0].grid()
a[0].plot(x, yelp, marker='.')
a[0].plot(x, imdb, marker='o')
a[0].plot(x, amazon, marker='^')
a[0].plot(x, snli, marker='v')
a[0].set_xlabel("$\lambda_{eps}$")
a[0].set_ylabel("s-BLEU")
a[0].legend(["Yelp", "IMDb", "Amazon", "SNLI"], loc ="lower right")

yelp = [float(e)*100 for e in "0.69	0.73	0.73	0.75	0.77	0.77".split()]
imdb = [float(e)*100 for e in "0.722	0.75	0.78	0.79	0.81	0.82".split()]
amazon = [float(e)*100 for e in "0.71	0.76	0.79	0.82	0.84	0.85".split()]
snli = [float(e)*100 for e in "0.49	0.49	0.52	0.56	0.63	0.67".split()]
a[1].plot(x, yelp, marker='.')
a[1].plot(x, imdb, marker='o')
a[1].plot(x, amazon, marker='^')
a[1].plot(x, snli, marker='v')
a[1].set_xlabel("$\lambda_{eps}$")
a[1].set_ylabel("Accuracy(\%)")
a[1].grid()

figure.savefig('./media/eps-effect.pdf')


In [ ]:
#preprocess data to suite tag and generate tsv form
import os

dataset = 'snli'
labels = ['0', '1']
splits = ['train', 'test', 'valid']
base = os.path.join('./data/annotated', dataset)
out = os.path.join(base, '{}-tag-gen.tsv'.format(dataset))
sents = []

stylemap = {'0':'neg', '1':'pos'}


with open(out, 'w') as f:
    f.write("txt\tsplit\tstyle\n") #format
    for split in splits:
        for lbl in labels:
            inpath = os.path.join(base, 'nli.{}.{}'.format(split, lbl))
            print("importing from {}".format(inpath))
            
            with open(inpath, 'r') as inp:
                for line in inp:
                    # print(line)
                    s = "val" if split == "valid" else split
                    l = line[:-1] + '\t' + s + '\t' + stylemap[lbl]
                    sents.append(l)
                    f.write(l + '\n')

In [42]:
#evaluate forms
from collections import Counter

dataset = 'snli'
scores = {'1':0, '2':0, '3':0, '4':0, 'na':0}

def get_maj(l):
    c = Counter(l)
    value1, count1 = c.most_common()[0]
    if(len(c.most_common()) == 1):
        return value1
    value2, count2 = c.most_common()[1]
    maj = -1 if count1 == count2 else value1
    return maj

def get_vals(path):
    vals = []
    with open(path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            words = line.split()
            if(len(words) != 0 and words[0] == "Decision:"):
                vals.append(int(words[1]))
    assert(len(vals) == 20)
    return vals

def print_results(reset=False):
    global scores
    print(scores)
    if(reset):
        scores = {'1':0, '2':0, '3':0, '4':0, 'na':0}
    return

people = ['pooja','sharan']
t = {p: None for p in people}
lbls = ['E', 'C']
ids = [e for e in range(5)]

def update(): #take 20 votes from each person and update votes for one file
    global t, scores
    for i in range(20):
        l = []
        for p in people:
            l.append(t[p][i])
        maj = get_maj(l)
        if(maj == -1):
            scores['na'] += 1
        else:
            scores[str(maj)] += 1
    

for src_lbl in lbls:
    for dst_lbl in lbls:
        if(src_lbl == dst_lbl):
            continue
        print('For {}to{} direction'.format(src_lbl, dst_lbl))
        base_dir = './human-evals/{}/{}to{}/'.format(dataset, src_lbl, dst_lbl)
        for id in ids:
            form = '{}to{}.{}'.format(src_lbl, dst_lbl, id)
            for p in people:
                form_path = './human-evals/{}/{}to{}/{}to{}.{}'.format(p, src_lbl, dst_lbl, src_lbl, dst_lbl, id)
                print(form_path)
                t[p] = get_vals(form_path)
            
            #after you get all the votes for everyone for a form, get results of that form
            update()
        #after {}to{} dir is over, print stats
        print_results(reset=True)   



For EtoC direction
./human-evals/pooja/EtoC/EtoC.0
./human-evals/sharan/EtoC/EtoC.0
./human-evals/pooja/EtoC/EtoC.1
./human-evals/sharan/EtoC/EtoC.1
./human-evals/pooja/EtoC/EtoC.2
./human-evals/sharan/EtoC/EtoC.2
./human-evals/pooja/EtoC/EtoC.3
./human-evals/sharan/EtoC/EtoC.3
./human-evals/pooja/EtoC/EtoC.4
./human-evals/sharan/EtoC/EtoC.4
{'1': 8, '2': 38, '3': 8, '4': 14, 'na': 32}
For CtoE direction
./human-evals/pooja/CtoE/CtoE.0
./human-evals/sharan/CtoE/CtoE.0
./human-evals/pooja/CtoE/CtoE.1
./human-evals/sharan/CtoE/CtoE.1
./human-evals/pooja/CtoE/CtoE.2
./human-evals/sharan/CtoE/CtoE.2
./human-evals/pooja/CtoE/CtoE.3
./human-evals/sharan/CtoE/CtoE.3
./human-evals/pooja/CtoE/CtoE.4
./human-evals/sharan/CtoE/CtoE.4
{'1': 3, '2': 31, '3': 1, '4': 13, 'na': 52}


In [28]:

l = [2,1,1,2]
l = [2,1,1,2]
c = Counter(l)
value1, count1 = c.most_common()[0]
value2, count2 = c.most_common()[1]

maj = -1 if count1 == count2 else value1
print(maj)

-1
